<a href="https://colab.research.google.com/github/D-girl-11t/8085_Simulator/blob/main/hw4_ds6992.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydes  #installing necessary package

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#importing necessary libraries
import math
import numpy as np
import scipy
import random
import csv
import pyDes
from itertools import *

In [ ]:
KEY = "Donttell"            #is the secret key, which is no more a secret after attack
cipher = pyDes.des(KEY)     #calling the des function from pydes package

plain_text = "security"


The cipher here accepts the 64 bit key (in byte form) , we are performing the attack with only one block of 64bit plain_text to avoid confusion, binary of "security" is 64bit.
It can be extended to any number of bits by choping the plain_text into 64bits blocks and iterating through each block

In [ ]:
print(dir(cipher))        #lets check all the attributes inside cipher

['DECRYPT', 'ENCRYPT', 'Kn', 'L', 'R', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_baseDes__key', '_des__BitList_to_String', '_des__String_to_BitList', '_des__create_sub_keys', '_des__des_crypt', '_des__expansion_table', '_des__fp', '_des__ip', '_des__left_rotations', '_des__p', '_des__pc1', '_des__pc2', '_des__permutate', '_des__sbox', '_guardAgainstUnicode', '_iv', '_mode', '_padData', '_padding', '_padmode', '_unpadData', 'block_size', 'crypt', 'decrypt', 'encrypt', 'final', 'getIV', 'getKey', 'getMode', 'getPadMode', 'getPadding', 'key_size', 'setIV', 'setKey', 'setMode', 'setPadMode', 'setPadding']


Below is the code for DES ALgorithm.
An attack is introduced at round 15. 
The function returns 3 values : ciphered text, Right and left blocks of 64bit before final permutation.

The left and right blocks can be retrieved by making the cipher text go through IP ( inverse of inverse permutation) and swapping the half of the 64bit blocks and breaking into two. Instead I just returned at the end of the function as our main focus is about attack on DES algorithm. 


The functions with cipher.function() is from pyDes package. We saw all the attributes pyDes has and I have used them accordingly everywhere

In [ ]:
#DES Algorithm for a block of 8bytes

def fault_encrypt(block, mode, fault_index):
  block = cipher._des__permutate(cipher._des__ip, block)
  L = block[:32]
  R = block[32:]

  # for 16 rounds
  for i in range(16):
    tempR = R[:]
    R = cipher._des__permutate(cipher._des__expansion_table, R)
    R = list(map(lambda x, y: x ^ y, R, cipher.Kn[i]))
    B = [R[:6], R[6:12], R[12:18], R[18:24], R[24:30], R[30:36], R[36:42], R[42:]]


    # Permutate B[1] to B[8] using the S-Boxes
    j = 0
    Bn = [0] * 32
    pos = 0
    while j < 8:
      # Work out the offsets
      m = (B[j][0] << 1) + B[j][5]
      n = (B[j][1] << 3) + (B[j][2] << 2) + (B[j][3] << 1) + B[j][4]

      # Find the permutation value
      v = cipher._des__sbox[j][(m << 4) + n]

      # Turn value into bits, add it to result: Bn
      Bn[pos] = (v & 8) >> 3
      Bn[pos + 1] = (v & 4) >> 2
      Bn[pos + 2] = (v & 2) >> 1
      Bn[pos + 3] = v & 1

      pos += 4
      j += 1

    R = cipher._des__permutate(cipher._des__p, Bn)


    # Xor with L[i - 1]  
    R = list(map(lambda x, y: x ^ y, R, L))
    L = tempR

    if(mode=="fault"):
      if(i==14):                #attack at round 15
        if R[fault_index] == 0:
          R[fault_index] =1
        else:
          R[fault_index] = 0


  # Final permutation of R[16]L[16]
  cipher.final = cipher._des__permutate(cipher._des__fp, R + L)
  return cipher.final, R, L

##I created a fault at ONCE for each bit of 32bit R15 to get 32 different ciphers.

In [ ]:
plain_text_inbits=[0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1]
fault_insertion = []    #just an array of numbers from 0 to 31 , each number is the fault number
for i in range(32):    #insert fault at particular indices
  fault_insertion.append(i)

for i in range(32):
  globals()['fault_'+str(i)]= fault_encrypt(plain_text_inbits, "fault", fault_insertion[i])
  globals()['fault_R16_'+str(i)] = globals()['fault_'+str(i)][1]        #32 fault R16 addressed using globals() variable
  globals()['fault_L16_'+str(i)] = globals()['fault_'+str(i)][2]         #32 fault L16 addressed using globals() variable

  globals()['original_'+str(i)] = fault_encrypt(plain_text_inbits, "original", fault_insertion[i])
  globals()['original_R16_'+str(i)] = globals()['original_'+str(i)][1]     #32 original R16 addressed using globals() variable
  globals()['original_L16_'+str(i)] = globals()['original_'+str(i)][2]      #32 original L16 addressed using globals() variable




In [ ]:
#lets just check all fault R16 for 32 different faults
for i in range(32):
  print( globals()['fault_R16_'+str(i)])

[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1]
[1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]
[1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1]
[1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1]
[1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1]
[1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1]
[1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1]
[1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1]
[1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1]
[1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1]
[1, 1, 1, 1, 0, 1, 1, 1, 1, 0,

In [ ]:
#lets check the difference between original L16 and fault L16. This tells us which bit is altered
for i in range(32):
  print(list(map(lambda x, y: x ^ y,  globals()['original_L16_'+str(i)],  globals()['fault_L16_'+str(i)])))


[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

#Theory

Lets focus on one particular SBox S with one particular faulted cypher. Let's calculate L16 ^ L16* for that SBox, that is 4 bits.
With P-1 being the inverse of the permutation P we have :




*   L16 = R15 and R16 = P(S(E(R15) ^ K16)) ^ L15
*   after introducing a fault in R15
– R15’ = R15 ^ e
– L16’ = R15’ 
and R16’ = P(S(E(R15’) ^ K16)) ^L15
* R16 ^ R16’=P(S(E(R15) ^K16)) ^ P(S(E(R15 ^e) ^K16)
* P-1(R16 ^ R16’) = S(E(R15) ^ K16) ^ S(E(R15*) ^ K16)






In [ ]:
# lets make all the combinations of 0 and 1 of length 6 
findkey=list(product([0,1],repeat=6))

In [ ]:
#sbox function
def sboxing(expanded):
  expanded =[expanded[:6], expanded[6:12], expanded[12:18], expanded[18:24], expanded[24:30], expanded[30:36], expanded[36:42], expanded[42:]]
  sbox_output=[]  
  for j in range(8):  #for every sbox
    Bn_poss = []
    for k in findkey: #for all different combinations of key
      Bn=[]
      B = list(map(lambda x, y: x ^ y, expanded[j], list(k) ))
      # print("box "+ str(j))

      m = (B[0] << 1) + B[5]
      n = (B[1] << 3) + (B[2] << 2) + (B[3] << 1) + B[4]

      # Find the permutation value
      v = cipher._des__sbox[j][(m << 4) + n]

      # Turn value into bits, add it to result: Bn
      Bn.append((v & 8) >> 3)
      Bn.append( (v & 4) >> 2)
      Bn.append((v & 2) >> 1)
      Bn.append( v & 1)

      Bn_poss.append(Bn)
    # print(len(Bn_poss))
    sbox_output.append(Bn_poss)
    # print(len(sbox_output))
  return sbox_output


In [ ]:
#performing step 3 in theory
Pinv = [
    9,  17, 23, 31,
    13, 28, 2,  18,
    24, 16, 30, 6,
    26, 20, 10, 1,
    8,  14, 25, 3,
    4,  29, 11, 19,
    32, 12, 22, 7,
    5,  27, 15, 21
]
for i in range(32):
  Pinv[i] = Pinv[i]-1
  
for i in range(32):
  #delta is the difference between original R16 and fault R16
  globals()['delta_'+str(i)]= list(map(lambda x, y: x ^ y,  globals()['original_R16_'+str(i)],  globals()['fault_R16_'+str(i)]))
  #delta goes through Pinverse 
  globals()['delta_'+str(i)] = cipher._des__permutate(Pinv, globals()['delta_'+str(i)])

  globals()['delta_compare'+str(i)] = [globals()['delta_'+str(i)][:4], globals()['delta_'+str(i)][4:8], 
                                       globals()['delta_'+str(i)][8:12], globals()['delta_'+str(i)][12:16], 
                                       globals()['delta_'+str(i)][16:20], globals()['delta_'+str(i)][20:24], 
                                       globals()['delta_'+str(i)][24:28], globals()['delta_'+str(i)][28:32]]
  print(globals()['delta_'+str(i)])

#expansion of orginal L16 and fault L16
  globals()['expanded_L16'+str(i)] = cipher._des__permutate(cipher._des__expansion_table, globals()['original_L16_'+str(i)])
  globals()['expanded_fault_L16'+str(i)] = cipher._des__permutate(cipher._des__expansion_table, globals()['fault_L16_'+str(i)])


[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1,

In [ ]:
# expanded_L16 = cipher._des__permutate(cipher._des__expansion_table, original_L16)
# expanded_fault_L16 = cipher._des__permutate(cipher._des__expansion_table, fault_L16)

# total_original=[]
# total_fault=[]
# for i in range(64):
#   full_original = []
#   full_fault =[]
#   for j in range(8):
#     full_original+=sboxing(expanded_L16)[j][i]
#     full_fault+=sboxing(expanded_fault_L16)[j][i]
#     # print(len(full))
#   original_perm = cipher._des__permutate(cipher._des__p, full_original)
#   fault_perm = cipher._des__permutate(cipher._des__p, full_fault)

#   total_original.append(original_perm)
#   total_fault.append(fault_perm)

  
# for i in range(5):
#   print(len(sboxing(globals()['expanded_L16'+str(i)])))


In [ ]:
poss=[]      #number of keys per iteration per sbox......just numbers 
keys_poss=[]  #contains index of findkey of corresponding sbox of corresponding iteration

for k in range(32):         #for each iteration
  possibilities_per_sbox=[]
  keysbox=[]
  for j in range(8):        #for each sbox
    kk=[]
    count=0
    original_sboxing_combinations = sboxing(globals()['expanded_L16'+str(k)])[j]    
    fault_sboxing_combinations = sboxing(globals()['expanded_fault_L16'+str(k)])[j]
  

    for i in range(64):     #for each possibilty of 6 bit key for sbox
      sxor = list(map(lambda x, y: x ^ y,original_sboxing_combinations[i], fault_sboxing_combinations[i] ))
      # print(sxor)
      if sxor == globals()['delta_compare'+str(k)][j]:
        count=count+1
        kk.append(i)
        # print("key at "+ str(i) + str(findkey[i])+ " for box " +str(j)+ str(sxor))
    keysbox.append(kk)
    # print(keysbox)
    possibilities_per_sbox.append(count)
  keys_poss.append(keysbox)

  # print(possibilities_per_sbox)
      
  poss.append(possibilities_per_sbox)
poss  #number of keys per iteration per sbox

[[6, 64, 64, 64, 64, 64, 64, 14],
 [8, 64, 64, 64, 64, 64, 64, 64],
 [6, 64, 64, 64, 64, 64, 64, 64],
 [4, 14, 64, 64, 64, 64, 64, 64],
 [10, 8, 64, 64, 64, 64, 64, 64],
 [64, 16, 64, 64, 64, 64, 64, 64],
 [64, 14, 64, 64, 64, 64, 64, 64],
 [64, 12, 4, 64, 64, 64, 64, 64],
 [64, 14, 14, 64, 64, 64, 64, 64],
 [64, 64, 10, 64, 64, 64, 64, 64],
 [64, 64, 10, 64, 64, 64, 64, 64],
 [64, 64, 12, 12, 64, 64, 64, 64],
 [64, 64, 12, 8, 64, 64, 64, 64],
 [64, 64, 64, 8, 64, 64, 64, 64],
 [64, 64, 64, 4, 64, 64, 64, 64],
 [64, 64, 64, 4, 8, 64, 64, 64],
 [64, 64, 64, 16, 6, 64, 64, 64],
 [64, 64, 64, 64, 8, 64, 64, 64],
 [64, 64, 64, 64, 10, 64, 64, 64],
 [64, 64, 64, 64, 6, 8, 64, 64],
 [64, 64, 64, 64, 4, 4, 64, 64],
 [64, 64, 64, 64, 64, 4, 64, 64],
 [64, 64, 64, 64, 64, 6, 64, 64],
 [64, 64, 64, 64, 64, 6, 6, 64],
 [64, 64, 64, 64, 64, 16, 8, 64],
 [64, 64, 64, 64, 64, 64, 6, 64],
 [64, 64, 64, 64, 64, 64, 8, 64],
 [64, 64, 64, 64, 64, 64, 12, 16],
 [64, 64, 64, 64, 64, 64, 4, 12],
 [64, 64, 

as we can see, for sbox1 , only iterations 0,1,2,3,4,31 shows possible variation of keys satisfy.

Lets combine them and get the index of the findkey of corresponding sbox

In [ ]:
# ******************not required*****************************
letc =[]
fault_number=[]
for i in range(8):
  arr=[]
  common=[]
  for j in range(32):
    arr.append(poss[j][i])
    if(poss[j][i] !=64):
      common.append(j)

  # print(arr)
  # print(min(arr))
  letc.append(common)
  fault_number.append(arr.index(min(arr)))
  # print(fault_number)
  

  print("for sbox "+ str(i)+" try keys in fault "+str(fault_number[i]))   #shows minimum key variations to be observed
print(letc)

for sbox 0 try keys in fault 3
for sbox 1 try keys in fault 4
for sbox 2 try keys in fault 7
for sbox 3 try keys in fault 14
for sbox 4 try keys in fault 20
for sbox 5 try keys in fault 20
for sbox 6 try keys in fault 28
for sbox 7 try keys in fault 30
[[0, 1, 2, 3, 4, 31], [3, 4, 5, 6, 7, 8], [7, 8, 9, 10, 11, 12], [11, 12, 13, 14, 15, 16], [15, 16, 17, 18, 19, 20], [19, 20, 21, 22, 23, 24], [23, 24, 25, 26, 27, 28], [0, 27, 28, 29, 30, 31]]


In [ ]:
kk=[]         #correct key index for corresponding sbox

for i in range(8):
  vliw=[]
  for j in letc[i]:
    vv=keys_poss[j][i]
    vliw = vliw+vv

  kk.append(max(set(vliw), key = vliw.count))   #This says maximum number of times an index is repeated and what it is
kk                                              #and we append in kk

#forgive my variable naming

[56, 27, 56, 38, 36, 28, 43, 56]

#The last round key

In [ ]:
final_round_key = []
for i in kk:
  final_round_key+= list(findkey[i])
print(final_round_key)

[1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]


In [ ]:
#using the package

cipher.encrypt(plain_text)
print(cipher.encrypt(plain_text))
print(cipher.Kn[15])

b'\xcdtE\x9c\xf4\xcca\xfb'
[1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]


In [ ]:
# lets check if we got the right subkey key
# cipher has an attribute, Kn which contains array of subkeys for 16 rounds

#by XORing the obtained key and cipher.Kn key , we can see that all values are zeroes, which means they are same
print(list(map(lambda x, y: x ^ y,final_round_key, cipher.Kn[15])))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:

# for i in range(8):
#   key_index = keys_poss[fault_number[i]][i]
#   print(key_index)
#   # print("possible sbox "+str(i)+" keys are")
#   arr=[]
#   for j in key_index:   
#     arr.append(findkey[j]) 
    
#   globals()['possbox_'+str(i)+'_keys'] = arr
#   # print( globals()['possbox_'+str(i)+'_keys'])

#Retrieving the "KEY"

In [ ]:
#This was a useful function, but....sadly not using it
# def bitstring_to_bytes(s):
#     v = int(s, 2)
#     b = bytearray()
#     while v:
#         b.append(v & 0xff)
#         v >>= 8
#     return bytes(b[::-1])

# s = "11100000111101000000110100111100101111011101000010011000110111"
# print(bitstring_to_bytes(s))

In [248]:
def get_bit_uint64_t_most(num, bit_pos):
    mask = 1 << (64 - bit_pos)                              #extracts a bit from a 64-bit unsigned integer (uint64_t) starting from the most significant bit (MSB).
    return 1 if num & mask else 0

def set_bit_uint64_t(num, bit_value, bit_pos):
    mask = 1 << (64 - bit_pos)            # sets a specific bit in a 64-bit unsigned integer (uint64_t).
    if bit_value == 1:
        num |= mask
    else:
        num &= ~mask
    return num

def build_C16_D16(K16):          #building C16 AND D16 FROM LAST ROUND KEY OBTAINED
    C0 = K16 >> 28
    D0 = K16 & 0xFFFFFFF
    return C0, D0

PC1 = [                                   #PC1 permutation through which key changes from 56 to 48
    57, 49, 41, 33, 25, 17, 9, 1,
    58, 50, 42, 34, 26, 18, 10, 2,
    59, 51, 43, 35, 27, 19, 11, 3,
    60, 52, 44, 36, 63, 55, 47, 39,
    31, 23, 15, 7, 62, 54, 46, 38,
    30, 22, 14, 6, 61, 53, 45, 37,
    29, 21, 13, 5, 28, 20, 12, 4
]

def build_K56(C0, D0):     #extracting 56 bit key 
    K56 = 0x00
    K56 = C0 << 28
    K56 |= D0
    return K56

def build_K(C16, D16):     #applies the PC1 permutation to K56 and returns the resulting 48-bit unsigned integer
    K = 0x00
    for i in range(56):
        rang = PC1[i]
        bit = get_bit_uint64_t_most(build_K56(C16, D16), i + 1 + 8)
        K = set_bit_uint64_t(K, bit, 65 - rang)
    return K

def set_parity_bits(K):           #sets the parity bits in the key K. It loops over each of the eight 7-bit groups of the key, counts the number of 1 bits in each group, 
    for i in range(8):                        #and sets the parity bit to 0 if the count is odd and 1 if the count is even
        j = 1 + i * 8
        compteur = sum(get_bit_uint64_t_most(K, j + k) for k in range(7))
        bit8 = 1 if compteur % 2 == 0 else 0
        K = set_bit_uint64_t(K, bit8, 8 * (8 - i) - 7)
    return K




#finding the key

below function performs an exhaustive search over the 8 bits of the key that are not known. 
Iterates over all possible values of these 8 bits and sets them in the key. 
For each possible key, it encrypts the plaintext using DES and compares the resulting ciphertext with the expected ciphertext. If the ciphertext matches the expected value, it returns the key. Otherwise, it continues searching.

In [249]:
class DATA:
  # k16: a hexadecimal number
  # key: initially set to zero, but will later be updated
  # message_clair: some variable plain_text is used to initialize this attribute
  # chiffre_juste: an integer value which appears to be a pre-determined ciphertext value, 
  # to be used as a reference for validating the correctness of the code's output.

    def __init__(self):
        self.k16 = 0xE1BE2691CAF8  # k16 in hex form
        self.key = 0              # key
        self.message_clair = plain_text   # plaintext in uint64_t format
        self.chiffre_juste = 14804534415497912827 # to get the correct output



def find_K(data: DATA) -> int:
    C16, D16 = build_C16_D16(data.k16)
    data.key = build_K(C16, D16)

    for i in range(0xFF):
        # recherche exhaustive sur les 8 bits non connus
        bit1 = (i >> 7) & 1
        data.key = set_bit_uint64_t(data.key, bit1, 7)
        bit2 = (i >> 6) & 1
        data.key = set_bit_uint64_t(data.key, bit2, 14)
        bit3 = (i >> 5) & 1
        data.key = set_bit_uint64_t(data.key, bit3, 46)
        bit4 = (i >> 4) & 1
        data.key = set_bit_uint64_t(data.key, bit4, 5)
        bit5 = (i >> 3) & 1
        data.key = set_bit_uint64_t(data.key, bit5, 50)
        bit6 = (i >> 2) & 1
        data.key = set_bit_uint64_t(data.key, bit6, 11)
        bit7 = (i >> 1) & 1
        data.key = set_bit_uint64_t(data.key, bit7, 51)
        bit8 = i & 1
        data.key = set_bit_uint64_t(data.key, bit8, 45)

        # ajout des bits de parité
        set_parity_bits(data.key)

        # data.message_clair = data.message_clair_original
        # print(key)
        b = cipher.encrypt(data.message_clair, data.key)
        n = int.from_bytes(b, byteorder='big')
        uint64_t_value = n & 0xffffffffffffffff
        # print(uint64_t_value)
        

        # Si le chiffrement du message clair avec la clé donne le chiffré correct alors la clé sur 64 bits a été trouvée.
        if uint64_t_value == data.chiffre_juste:
            # print(uint64_t_value)
            return key

    return "Try again!"

In [250]:
# The find_K() function takes an instance of the DATA class and tries to find the key 
# by brute-forcing the 8 unknown bits. It sets each bit of the key to a possible value and 
# checks if the encrypted message matches the correct ciphertext.

# If the correct key is found, the function returns 0. Otherwise, it returns 1. 
# The code also prints out the key and the encrypted message for each iteration.

objectcall = DATA()
print(find_K(objectcall))

Donttell
